In [ ]:
# Instala Surprise si es necesario:
# pip install scikit-surprise

import os
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import GridSearchCV
from surprise import accuracy

# 1. Carga de datos procesados
processed_dir = 'data/processed'
train_df = pd.read_csv(os.path.join(processed_dir, 'train.csv'))
test_df  = pd.read_csv(os.path.join(processed_dir, 'test.csv'))

# 2. Prepara el dataset para Surprise usando solo train_df
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[['user_id', 'book_id', 'rating']], reader)

# 3. Definición de la grid de parámetros
param_grid = {
    'k': [10, 20, 50],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [True, False]
    }
}

# 4. Búsqueda de los mejores parámetros con validación cruzada (5 folds)
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

print("Mejor RMSE en CV:", gs.best_score['rmse'])
print("Mejores parámetros:", gs.best_params['rmse'])

# 5. Entrenamiento final con esos parámetros
algo = KNNBasic(**gs.best_params['rmse'])
trainset = data.build_full_trainset()
algo.fit(trainset)

# 6. Preparación de los datos de test para Surprise
testset = list(zip(
    test_df['user_id'].values,
    test_df['book_id'].values,
    test_df['rating'].values
))

# 7. Evaluación en test
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae  = accuracy.mae(predictions)

print(f"KNN colaborativo → RMSE: {rmse:.4f}, MAE: {mae:.4f}")
